In [1]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import boto3
import json

ModuleNotFoundError: No module named 'dsgreen'

In [ ]:
Target_year = 2022

pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:,.1f}'.format

out_folder = 'Output'
AWS_folder = "../MAP/AWS/Latest/"
data_folder = 'Output\Ecount'
src_folder = "Ecount_download"
src_folder = os.path.join(src_folder,str(Target_year))
src_folder = os.path.join(src_folder,'inv')